# **저시력자를 위한 원화 화폐 분류**
---
- 본 과제는 UltraLytics YOLO v5 모델 사용을 권장합니다.
    - 본 파일의 목차는 UltraLytics YOLO v5에 맞게 작성되어 있습니다.
    - 다른 모델을 찾아서 사용하셔도 좋습니다.
    - 산출물이 잘 나오면 됩니다 : )
---

## 0.미션
---
- **과제 수행 목표**
    - 본 과제는 Object Detection 문제입니다.
    - Object Detection 문제로 접근하기 위해 **데이터셋 전처리**를 하셔야 합니다.
    - 데이터셋 : money_dataset.zip
        1. 데이터셋은 압축 파일로 제공됩니다.
        2. 압축 파일 안에는 화폐마다 폴더가 개별적으로 존재합니다.
        3. 폴더 안에는 화폐 이미지와 화폐 정보가 담긴 json 파일이 있습니다.
    - 여러분이 직접 촬영한 화폐 사진들을 탐지 과정에서 이용 해보세요.
    - 이미지에 화폐 하나만 나오게 촬영하는 것은 지양해주세요.
    - 다양한 방법으로 화폐를 촬영하고 결과를 확인해보세요.
        - ex 1) 화폐의 모든 종류를 한 이미지에 나오게 촬영
        - ex 2) 여러 화폐를 겹치게 하여 촬영
---
- **Key Point**
    1. 모델에 맞는 폴더 구조 확인
    2. 이미지 축소 비율에 맞춰 좌표값 변경
        - 좌표를 이미지 리사이즈한 비율로 변경
    3. 모델에 맞는 정보 추출/형식 변경
        - json 파일에서 정보 추출 및 모델 형식에 맞게 변경
    4. 화폐당 하나의 클래스로 변경
        - 총 8개 클래스
    5. 모델 선택 필요
---

## 1.환경설정

### (1) 구글 드라이브 연동, 데이터 다운로드
---
- 아래의 코드 셀을 반드시 실행시켜야 합니다.
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!pip install gdown

### (2) 데이터셋 불러오기
---
- **세부요구사항**
    - 데이터셋 파일의 압축을 해제하세요.
---
- 예제 코드에서는 zipfile 모듈을 이용하였습니다.
    - [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
import zipfile, gdown,os
url ="https://drive.google.com/file/d/1k1tXDK35s6BsMTPGWSl5GVGNoPfC898X/view?usp=drive_link"
file_name = "money_dataset.zip"
output = "/content/drive/MyDrive/project_0921/" + file_name # 변경 가능
if not os.path.exists(output):
    gdown.download(url=url, output=output, quiet=False, fuzzy=True)

In [ ]:
# 데이터셋 압축 파일 경로 : 유저별로 상이할 수 있음
money_data = zipfile.ZipFile(output)

In [ ]:
# 데이터셋 압축 해제
money_data.extractall('Dataset')

## 2.데이터 전처리

### (1) 폴더 구조 생성 및 파일 이동
---
- **세부요구사항**
    -  모델에서 요구하는 폴더 구조를 만들어야 합니다.
        - Hint : Image와 Label을 구분하는 폴더를 만들어 주세요
---
- 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
# 1.폴더 구조 만들기
!mkdir /content/Dataset/images;
!mkdir /content/Dataset/images/train; mkdir /content/Dataset/images/val

!mkdir /content/Dataset/labels;
!mkdir /content/Dataset/labels/train; mkdir /content/Dataset/labels/val

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import glob, shutil

In [ ]:
# 2. Dataset metadata 입력
won_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']
data_path = '/content/Dataset/'

---
- 데이터를 Training set | Validation set으로 분할하세요.
    - 예시 : Training과 Validation은 8:2로 분리
- Hint : 이미지 데이터는 /images에, JSON 데이터는 /labels에 넣어주세요
    - 예시 : /dataset/images/train, /dataset/labels/train
    - 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)

    ※ 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
    
---

In [ ]:
# 3. 데이터를 Training set | Validation set으로 분할하세요.
images = glob.glob(data_path + '*/*.jpg')
labels = glob.glob(data_path + '*/*.json')
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=2023)

In [ ]:
"/".join(x_train[0].split('/')[:-1])

'/content/Dataset/10'

In [ ]:
import shutil

# x_train 복사
target_folder_1 = '/content/Dataset/images/train'  # 대상 폴더 경로 (원하는 폴더 경로로 변경)
target_folder_2 = '/content/Dataset/images/val'
target_folder_3 = '/content/Dataset/labels/train'
target_folder_4 = '/content/Dataset/labels/val'

# 파일을 대상 폴더로 복사
for file_name in x_train:
    destination_path = os.path.join(target_folder_1, file_name.split('/')[-1])  # 대상 폴더로 복사할 경로
    shutil.copyfile(file_name, destination_path)

for file_name in x_val:
    destination_path = os.path.join(target_folder_2,  file_name.split('/')[-1])  # 대상 폴더로 복사할 경로
    shutil.copyfile(file_name, destination_path)

for file_name in y_train:
    destination_path = os.path.join(target_folder_3,  file_name.split('/')[-1])  # 대상 폴더로 복사할 경로
    shutil.copyfile(file_name, destination_path)

for file_name in y_val:
    destination_path = os.path.join(target_folder_4,  file_name.split('/')[-1])  # 대상 폴더로 복사할 경로
    shutil.copyfile(file_name, destination_path)

### (2) json에서 정보 추출
---
- **세부요구사항**
    - json 파일에서 필요한 정보를 추출하세요:
        - 위치 정보 : x1, x2, y1, y2
        - 박스 정보 : shape_type
        - 클래스 정보 : labels
    - 화폐당 하나의 클래스로 변경하세요.
        - json 파일에는 화폐 클래스가 앞뒷면으로 구분되어 있습니다.
        - 화폐의 앞뒷면 구분을 없애주세요.
            - 예시 : 'ten_front', 'ten_back' -> 'ten'
    - 화폐의 위치 정보를 YOLO 모델 형식에 맞게 변경 해주세요.
        - 사용되는 이미지는 원본에서 1/5로 축소되어 있습니다.
        - json 파일의 정보는 원본 기준 데이터이므로 위치 정보 추출을 할 때 x값과 y값을 1/5로 줄여주세요.
    - 이렇게 변경된 정보를 YOLO label 형식에 맞게 txt파일로 저장 해 주세요.
        - Hint : YOLO Labeling Format [label, x-center, y-center, width-norm, height-norm]
---

In [ ]:
import os, json

In [ ]:
json_path = '/content/Dataset/labels/'
temp_list = ['train', 'val']

In [ ]:
########################
# 이 셀부터 코드 작성하세요
# Json 파일에서 필요한 정보만 골라 txt로 바꾸는 작업임을 기억하세요!
#######################
x_centers = []
y_centers = []
classes = []
widths = []
heights = []
file_names = []

train_file_names = []
val_file_names = []

In [ ]:
for i in temp_list:
    if i == 'train':
        train_file_names = glob.glob(json_path+i+'/*.json')
    else:
        val_file_names = glob.glob(json_path+i+'/*.json')

In [ ]:
train_file_names.sort()
val_file_names.sort()

In [ ]:
train_file_names[:5]

['/content/Dataset/labels/train/10000_B_DESK_0_1.json',
 '/content/Dataset/labels/train/10000_B_DESK_0_10.json',
 '/content/Dataset/labels/train/10000_B_DESK_0_100.json',
 '/content/Dataset/labels/train/10000_B_DESK_0_101.json',
 '/content/Dataset/labels/train/10000_B_DESK_0_102.json']

In [ ]:
val_file_names[:5]

['/content/Dataset/labels/val/10000_B_DESK_0_111.json',
 '/content/Dataset/labels/val/10000_B_DESK_0_121.json',
 '/content/Dataset/labels/val/10000_B_DESK_0_126.json',
 '/content/Dataset/labels/val/10000_B_DESK_0_131.json',
 '/content/Dataset/labels/val/10000_B_DESK_0_135.json']

In [ ]:
train_file_names[0].split('.json')[0]

'/content/Dataset/labels/train/10000_B_DESK_0_1'

In [ ]:
def convert_to_txt(file_name,label,min_xy,max_xy,image_width,image_height):
    file_name = file_name.split('.json')[0]+'.txt'
    label = label.split('_back')[0].split('_front')[0]
    x_min,y_min = min_xy
    x_max,y_max = max_xy

    if label == 'Ten':
        label = 0
    elif label == 'Fifty':
        label = 1
    elif label == 'Hundred':
        label = 2
    elif label == 'Five_Hundred':
        label = 3
    elif label == 'Thousand':
        label = 4
    elif label == 'Five_Thousand':
        label = 5
    elif label == 'Ten_Thousand':
        label = 6
    elif label == 'Fifty_Thousand':
        label = 7

    width = x_max - x_min
    height = y_max - y_min
    width_norm = round(width / image_width,7)
    height_norm = round(height / image_height,7)

    center_x = round(((x_min + x_max) / 2) / image_width,7)
    center_y = round(((y_min + y_max) / 2) / image_height,7)

    x_centers.append(center_x)
    y_centers.append(center_y)
    classes.append(label)
    widths.append(width_norm)
    heights.append(height_norm)
    file_names.append(file_name)

In [ ]:
########################
# 이 셀부터 코드 작성하세요
# Json 파일에서 필요한 정보만 골라 txt로 바꾸는 작업임을 기억하세요!
########################
x_centers = []
y_centers = []
classes = []
widths = []
heights = []
file_names = []

for i in train_file_names:
    with open(i,'r') as f:
        data = json.load(f)
        convert_to_txt(i,
                    data['shapes'][0]['label'],
                    data['shapes'][0]['points'][0],
                    data['shapes'][0]['points'][1],
                    data['imageWidth'],
                    data['imageHeight'])

num = len(file_names)
#name = '/content/Dataset/labels/train/'
print(num)
for i in range(num):
    fn = file_names[i]
    with open(fn,'w+') as f:
        x = x_centers[i]
        y = y_centers[i]
        w = widths[i]
        h = heights[i]
        cl = classes[i]

        f.write(str(cl)+' '+str(x)+' '+str(y)+' '+str(w)+' '+str(h))

4174


In [ ]:
text_file = glob.glob('/content/Dataset/labels/train/*.txt')
text_file[:5]

['/content/Dataset/labels/train/500_1113_9.txt',
 '/content/Dataset/labels/train/50000_F_STUFF_0_16.txt',
 '/content/Dataset/labels/train/50000_F_DESK_0_118.txt',
 '/content/Dataset/labels/train/500_792_1.txt',
 '/content/Dataset/labels/train/10_6914_1.txt']

In [ ]:
########################
# 이 셀부터 코드 작성하세요
# Json 파일에서 필요한 정보만 골라 txt로 바꾸는 작업임을 기억하세요!
########################
x_centers = []
y_centers = []
classes = []
widths = []
heights = []
file_names = []

for i in val_file_names:
    with open(i,'r') as f:
        data = json.load(f)
        convert_to_txt(i,
                    data['shapes'][0]['label'],
                    data['shapes'][0]['points'][0],
                    data['shapes'][0]['points'][1],
                    data['imageWidth'],
                    data['imageHeight'])

num = len(file_names)
print(num)
#name = '/content/Dataset/labels/val/'

for i in range(num):
    fn = file_names[i]
    with open(fn,'w+') as f:
        x = x_centers[i]
        y = y_centers[i]
        w = widths[i]
        h = heights[i]
        cl = classes[i]

        f.write(str(cl)+' '+str(x)+' '+str(y)+' '+str(w)+' '+str(h))

1044


In [ ]:
text_file = glob.glob('/content/Dataset/labels/val/*.txt')
text_file[:5]

['/content/Dataset/labels/val/500_556_1.txt',
 '/content/Dataset/labels/val/1000_B_DESK_0_69.txt',
 '/content/Dataset/labels/val/1000_F_STUFF_0_47.txt',
 '/content/Dataset/labels/val/10_1274_9.txt',
 '/content/Dataset/labels/val/50000_F_HAND_0_7.txt']

### (3) 데이터셋 정보가 담긴 파일 생성
---
- **세부요구사항**
    - 파일 안에 있어야 할 정보는 아래와 같습니다.
        - 학습할 클래스 이름 정보
        - 학습할 클래스 수 정보
        - Training, Validation 데이터셋 위치 정보
---
- 가장 대중적으로 이용하는 라이브러리는 yaml 입니다.
    - [yaml document](https://pyyaml.org/wiki/PyYAMLDocumentation)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
import yaml

won_dict = {0:'10', 1:'50', 2:'100', 3:'500', 4:'1000', 5:'5000', 6:'10000',7:'50000'}

yaml_data = {
        "path":"/content/Dataset",
        "train":"images/train/",
        "val":"images/val/",
        "nc":8,
        "names":won_dict
    }

print(yaml.dump(yaml_data))

names:
  0: '10'
  1: '50'
  2: '100'
  3: '500'
  4: '1000'
  5: '5000'
  6: '10000'
  7: '50000'
nc: 8
path: /content/Dataset
train: images/train/
val: images/val/



In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
with open('/content/Dataset/money.yaml', 'w') as f :
    yaml.dump(yaml_data, f)

## 3.모델링

### (1) 모델 라이브러리 설치
---

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
!pip install -r yolov5/requirements.txt  # install

fatal: destination path 'yolov5' already exists and is not an empty directory.


### (2) 가중치 파일 다운로드
---
- **세부요구사항**
    - 모델 개발자가 제공하는 사전 학습 가중치 파일을 다운로드 하세요.
        - 해당 과정이 불필요하다면 넘어가셔도 됩니다!
---

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
!python /content/yolov5/detect.py --weights yolov5n.pt

detect: weights=['yolov5n.pt'], source=yolov5/data/images, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
image 1/2 /content/yolov5/data/images/bus.jpg: 640x480 4 persons, 1 bus, 47.8ms
image 2/2 /content/yolov5/data/images/zidane.jpg: 384x640 2 persons, 1 tie, 49.2ms
Speed: 0.6ms pre-process, 48.5ms inference, 1.3ms NMS per image at shape (1, 3, 640, 640)
Results saved to yolov5/runs/detect/exp6


### (3) 학습 : train.py
---
- **세부요구사항**
    - UltraLytics YOLO v5에는 아래의 데이터가 필요합니다.
        - 데이터셋 정보가 담긴 yaml 파일
        - 사용하려는 모델 구조에 대한 yaml 파일
        - 사용하려는 모델의 가중치 파일
---

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
!python /content/yolov5/train.py --epochs 20 --data /content/Dataset/money.yaml --cfg /content/yolov5/models/yolov5n.yaml --weights /content/yolov5/yolov5n.pt --name yolov5_coco

train: weights=/content/yolov5/yolov5n.pt, cfg=/content/yolov5/models/yolov5n.yaml, data=/content/Dataset/money.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=yolov5_coco, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.

## 4.탐지 : detect.py
---
- **세부요구사항**
    - 학습 과정에서 생성된 가중치 파일을 이용하세요.
    - IoU threshold를 0.25 이하로 설정하세요.
    - confidence threshold를 0.75 이상으로 설정하세요.
---
- 여러분이 **직접 촬영한 화폐 사진과 동영상**을 탐지 과정에 이용하여 결과를 확인하세요.
    - 조건
        1. 화폐의 수를 늘려가며 촬영 해보세요.
            - ex) 50원 하나, 50원 둘, 50원 셋, ...
        2. 화폐의 종류를 늘려가며 촬영 해보세요.
            - ex) 50원 하나와 100원 하나, 50원 하나와 100원 하나와 1000원 하나, ...
        3. 사진은 최소 30장 이상, 동영상은 최소 하나 이상 촬영하여 사용 해보세요.
---

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/yolov5_coco/weights/best.pt --source /content/drive/MyDrive/project_0921/datas/

detect: weights=['/content/yolov5/runs/train/yolov5_coco/weights/best.pt'], source=/content/drive/MyDrive/project_0921/datas/, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5n summary: 157 layers, 1769989 parameters, 0 gradients, 4.2 GFLOPs
image 1/12 /content/drive/MyDrive/project_0921/datas/Fifty_01.jpg: 640x640 1 50, 6.1ms
image 2/12 /content/drive/MyDrive/project_0921/datas/Fifty_thousand_01.jpg: 640x384 1 50000, 48.1ms
image 3/12 /content/drive/MyDrive/project_0921/datas/Fifty_thousand_02.jpg: 384x6

In [ ]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/yolov5_coco/weights/best.pt --iou-thres=0.25 --conf 0.75 --source /content/drive/MyDrive/project_0921/datas/

detect: weights=['/content/yolov5/runs/train/yolov5_coco/weights/best.pt'], source=/content/drive/MyDrive/project_0921/datas/, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.75, iou_thres=0.25, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5n summary: 157 layers, 1769989 parameters, 0 gradients, 4.2 GFLOPs
image 1/12 /content/drive/MyDrive/project_0921/datas/Fifty_01.jpg: 640x640 1 50, 5.5ms
image 2/12 /content/drive/MyDrive/project_0921/datas/Fifty_thousand_01.jpg: 640x384 1 50000, 45.6ms
image 3/12 /content/drive/MyDrive/project_0921/datas/Fifty_thousand_02.jpg: 384x6

In [ ]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/yolov5_coco/weights/best.pt --source 'https://youtu.be/NGen9lbU0NU'

Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/content/yolov5/utils/general.py", line 1115, in <module>
    if Path(inspect.stack()[0].filename).parent.parent.as_posix() in inspect.stack()[-1].filename:
  File "/usr/lib/python3.10/inspect.py", line 1673, in stack
    return getouterframes(sys._getframe(1), context)
  File "/usr/lib/python3.10/inspect.py", line 1650, in getouterframes
    frameinfo = (frame,) + getframeinfo(frame, context)
  File "/usr/lib/python3.10/inspect.py", line 1624, in getframeinfo
    lines, lnum = findsource(frame)
  File "/usr/lib/python3.10/inspect.py", line 952, in findsource
    module = getmodule(